# Generative AI Utilities

This repo contains utility functions for using Generative AI models.

Broadly it contains functionality for:
1. Running prompts through LLMs
2. Parsing the json output from LLMs

Currently it only supports Gemini, but over time it can support other models.

In [1]:
from pprint import pp

from genai_utils.gemini import run_prompt
from genai_utils.parsing import parse_model_json_output

### Basic Usage

Here's a simple demo...

In [2]:
test_prompt = """
Generate a family tree for the Tudor monarchs of England. Output should be in correctly formatted json.
""".strip()

output = run_prompt(test_prompt)
parsed = parse_model_json_output(output)
pp(parsed)

{'family_tree': {'name': 'Tudor Dynasty',
                 'description': 'The Tudor monarchs of England and their '
                                'immediate family, from Henry VII to Elizabeth '
                                'I.',
                 'people': [{'id': 'henry_vii',
                             'name': 'Henry VII',
                             'gender': 'male',
                             'birth': '1457',
                             'death': '1509',
                             'title': 'King of England',
                             'reign': '1485-1509',
                             'parents': [],
                             'spouses': ['elizabeth_of_york'],
                             'children': ['arthur_tudor',
                                          'margaret_tudor',
                                          'mary_tudor_france',
                                          'henry_viii']},
                            {'id': 'elizabeth_of_york',
                 

### Multimodal

It can also be used to process videos.

In [3]:
video = "gs://raphael-test/tiktok/7149378297489558830.mp4"

output = run_prompt("Summarise this video", video_uri=video)

pp(output)

('The video displays a static image of a pyramid titled "The Pyramid of '
 'Intellect." The pyramid is structured with educational degrees, from "High '
 'School Diploma" at the base, progressing upwards through "Associate\'s '
 'Degree," "Bachelor\'s Degree," "Master\'s Degree," and "Ph.D." However, at '
 'the very apex of the pyramid, above the Ph.D. level, is a label stating: '
 '"People who never took the Covid vaccine even after all the social '
 'pressure."\n'
 '\n'
 'Above the pyramid, text with American flag emojis reads: "Where are my pure '
 'blood brothers and sisters!!!"\n'
 '\n'
 'Background music plays throughout the video, with a female voice '
 'singing/rapping, though the lyrics are mostly indistinct. The meme suggests '
 'that refusing the COVID-19 vaccine, despite social pressure, is the ultimate '
 'sign of intellect, surpassing even advanced academic degrees.')


### Structured output

You can also get structured output from Gemini by specifying an output schema.

To define the structure, you just need to make a pydantic `BaseModel` and give that to `run_prompt`.
The types and descriptions will also be passed along with the keys for your json.
In this case we've passed along `list[Monarch]`, because we want a list of `Monarch` dictionaries to be returned.

In [4]:
from pydantic import BaseModel, Field

test_prompt = """
List all of the Stewart monarchs of England.
""".strip()


class Monarch(BaseModel):
    birth_name: str = Field(description="Birth name of the monarch")
    monarch_name: str = Field(description="Name as monarch")
    reign_start: str = Field(description="First year of reign")
    reign_end: str = Field(description="Final year of reign")
    spouse: list[str] = Field(description="A list of their spouses")
    children: list[str] = Field(description="A list of their children")


output = run_prompt(test_prompt, output_schema=list[Monarch])
parsed = parse_model_json_output(output)
pp(parsed)

[{'birth_name': 'James Charles Stuart',
  'monarch_name': 'James I',
  'reign_start': '1603',
  'reign_end': '1625',
  'spouse': ['Anne of Denmark'],
  'children': ['Henry Frederick',
               'Elizabeth Stuart',
               'Charles I',
               'Robert Stuart',
               'Mary Stuart',
               'Sophia Stuart']},
 {'birth_name': 'Charles Stuart',
  'monarch_name': 'Charles I',
  'reign_start': '1625',
  'reign_end': '1649',
  'spouse': ['Henrietta Maria of France'],
  'children': ['Charles II',
               'Mary, Princess Royal',
               'James II',
               'Elizabeth Stuart',
               'Henry Stuart, Duke of Gloucester',
               'Henrietta Anne Stuart']},
 {'birth_name': 'Charles Stuart',
  'monarch_name': 'Charles II',
  'reign_start': '1660',
  'reign_end': '1685',
  'spouse': ['Catherine of Braganza'],
  'children': []},
 {'birth_name': 'James Stuart',
  'monarch_name': 'James II',
  'reign_start': '1685',
  'reign_end': '168

### System Instructions

You can define a system instruction which basically gets added to the front of the prompt.
This is good for defining the desired behaviour of the model.

In [5]:
prompt = "Write a one paragraph review of Jaws."
instruction = (
    "You are a film critic."
    "You hate Hollywood films."
    "And you constantly mention your pet Badger."
)

output = run_prompt(prompt, system_instruction=instruction)
pp(output)

("Ah, *Jaws*. Another overblown Hollywood spectacle, isn't it? They trot out a "
 'rubber shark, some plucky small-town types, and expect us to be on the edge '
 "of our seats. Frankly, the 'terror' is about as convincing as a politician's "
 "promise; I've seen more genuine menace in Bartholomew's eyes when he's "
 'contemplating a particularly stubborn root in the garden. The whole affair '
 "feels engineered for maximum box office, a simplistic 'man vs. beast' "
 'narrative devoid of any real psychological depth or artistic merit. Give me '
 'a quiet evening with Bartholomew, observing the intricate politics of the '
 'compost heap, over this manufactured thrill ride any day.')


### Grounding

You can get Gemini to do a Google search, which it will then use to make its response.
This means you can access more up to data information, get a more factual response, and get citations.
It will be a significantly more expensive query though.

You can control this using the `use_grounding` and `inline_citations` arguments of `run_prompt`.
Inline citations only work if `use_grounding=True`.
Currently grounding does not work with structured output, so you must choose between grounding and structured output.

In [6]:
prompt = (
    "Who won the Battle of Crécy, and what was it's historical significance?"
    "Answer in one paragraph."
)

result = run_prompt(prompt, use_grounding=True, inline_citations=True)
pp(result)

("The Battle of Crécy, fought on August 26, 1346, during the Hundred Years' "
 'War, was decisively won by the English army led by King Edward III against a '
 'larger French force commanded by King Philip VI.  '
 '([1](https://www.ebsco.com/research-starters/history/battle-crecy), '
 '[2](https://www.britannica.com/event/Battle-of-Crecy), '
 '[3](https://en.wikipedia.org/wiki/Battle_of_Cr%C3%A9cy)) This victory held '
 'immense historical significance, primarily due to the devastating '
 'effectiveness of the English longbow, which allowed English archers to '
 'overwhelm the French cavalry and crossbowmen.  '
 '([1](https://www.ebsco.com/research-starters/history/battle-crecy), '
 '[4](https://www.berkhamstedcastle.org.uk/histories/1346-the-battle-of-crecy/), '
 '[5](https://www.llantrisant.net/index.php/history/battle-of-crecy), '
 '[6](https://www.history.co.uk/)) The battle marked a turning point in '
 'European warfare, signaling the decline of the traditional mounted knight '
 '